In [ ]:
!git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 98107, done.
remote: Counting objects: 100% (620/620), done.
remote: Compressing objects: 100% (368/368), done.
remote: Total 98107 (delta 292), reused 524 (delta 234), pack-reused 97487 (from 1)
Receiving objects: 100% (98107/98107), 621.63 MiB | 16.85 MiB/s, done.
Resolving deltas: 100% (71270/71270), done.
Updating files: 100% (3871/3871), done.


In [ ]:
%cd models/research/audioset/vggish/

/content/models/research/audioset/vggish


In [ ]:
"""
This notebook demonstrates how to replicate converting tensorflow
weights from tensorflow's vggish to torchvggish
"""

# Download the audioset directory using subversion
# !apt-get -qq install subversion   # uncomment if on linux
!svn checkout https://github.com/tensorflow/models/trunk/research/audioset

# Download audioset requirements
!pip install numpy scipy
!pip install resampy tensorflow six soundfile

# grab the VGGish model checkpoints & PCA params
!curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
!curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz

/bin/bash: line 1: svn: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.3 MB/s eta 0:00:00
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277M  100  277M    0     0  88.5M      0  0:00:03  0:00:03 --:--:-- 88.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73020  100 73020    0     0   266k      0 --:--:-- --:--:-- --:--:--  267k


In [ ]:
# Test install
# !mv audioset/* .
from vggish_smoke_test import *


Testing your install of VGGish

Resampling via resampy works!
Log Mel Spectrogram example:  [[-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 ...
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]
 [-4.48313252 -4.27083405 -4.17064267 ... -4.60069383 -4.60098887
  -4.60116305]]


/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:318: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '


VGGish embedding:  [-2.72986382e-01 -1.80314153e-01  5.19921184e-02 -1.43571526e-01
 -1.04673728e-01 -4.96598154e-01 -1.75267965e-01  4.23147976e-01
 -8.22126150e-01 -2.16801405e-01 -1.17509276e-01 -6.70077026e-01
  1.43174574e-01 -1.44183934e-01  8.73491913e-03 -8.71972442e-02
 -1.84393525e-01  5.96655607e-01 -3.43809605e-01 -5.79104424e-02
 -1.65071294e-01  4.22911644e-02 -2.55293399e-01 -2.36356765e-01
  1.80295616e-01  3.02612185e-01  1.08356833e-01 -4.48398024e-01
  1.22757629e-01 -2.99955189e-01 -5.55934191e-01  5.05966544e-01
  2.05210358e-01  8.87591839e-01  9.03702497e-01 -2.10566416e-01
 -3.27462405e-02  1.38691410e-01 -2.27416530e-01  1.14804000e-01
  5.95410109e-01 -4.76971269e-01  2.28232622e-01  1.54627025e-01
  1.64934218e-01  7.19252825e-01  1.24101830e+00  5.61996222e-01
  2.73531973e-01  3.09788287e-02  2.10977703e-01 -6.09551668e-01
 -3.15282375e-01  1.76392645e-01 -8.96190405e-02 -4.26822364e-01
  3.12993884e-01 -1.56592295e-01  3.31673503e-01  1.29436389e-01
  1.66

In [ ]:
import tensorflow as tf
import vggish_slim

vggish_dict = {}
# load the model and get info
with tf.Graph().as_default(), tf.compat.v1.Session() as sess:
    vggish_slim.define_vggish_slim(training=True)
    vggish_slim.load_vggish_slim_checkpoint(sess,"vggish_model.ckpt")

    tvars = tf.compat.v1.trainable_variables()
    tvars_vals = sess.run(tvars)

    for var, val in zip(tvars, tvars_vals):
        print("%s" % (var.name))
        print("\t" + str(var.shape))
        vggish_dict[var.name] = val
    print("values written to vggish_dict")

vggish/conv1/weights:0
	(3, 3, 1, 64)
vggish/conv1/biases:0
	(64,)
vggish/conv2/weights:0
	(3, 3, 64, 128)
vggish/conv2/biases:0
	(128,)
vggish/conv3/conv3_1/weights:0
	(3, 3, 128, 256)
vggish/conv3/conv3_1/biases:0
	(256,)
vggish/conv3/conv3_2/weights:0
	(3, 3, 256, 256)
vggish/conv3/conv3_2/biases:0
	(256,)
vggish/conv4/conv4_1/weights:0
	(3, 3, 256, 512)
vggish/conv4/conv4_1/biases:0
	(512,)
vggish/conv4/conv4_2/weights:0
	(3, 3, 512, 512)
vggish/conv4/conv4_2/biases:0
	(512,)
vggish/fc1/fc1_1/weights:0
	(12288, 4096)
vggish/fc1/fc1_1/biases:0
	(4096,)
vggish/fc1/fc1_2/weights:0
	(4096, 4096)
vggish/fc1/fc1_2/biases:0
	(4096,)
vggish/fc2/weights:0
	(4096, 128)
vggish/fc2/biases:0
	(128,)
values written to vggish_dict


In [ ]:
# Define torch model for vggish

import torch
import torch.nn as nn
import numpy as np

# From vggish_slim:
# The VGG stack of alternating convolutions and max-pools.
#     net = slim.conv2d(net, 64, scope='conv1')
#     net = slim.max_pool2d(net, scope='pool1')
#     net = slim.conv2d(net, 128, scope='conv2')
#     net = slim.max_pool2d(net, scope='pool2')
#     net = slim.repeat(net, 2, slim.conv2d, 256, scope='conv3')
#     net = slim.max_pool2d(net, scope='pool3')
#     net = slim.repeat(net, 2, slim.conv2d, 512, scope='conv4')
#     net = slim.max_pool2d(net, scope='pool4')
#     # Flatten before entering fully-connected layers
#     net = slim.flatten(net)
#     net = slim.repeat(net, 2, slim.fully_connected, 4096, scope='fc1')
#     # The embedding layer.
#     net = slim.fully_connected(net, params.EMBEDDING_SIZE, scope='fc2')

vggish_list = list(vggish_dict.values())
def param_generator():
    param = vggish_list.pop(0)
    transposed = np.transpose(param)
    to_torch = torch.from_numpy(transposed)
    result = torch.nn.Parameter(to_torch)
    yield result

class VGGish(nn.Module):
    def __init__(self):
        super(VGGish, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2))
        self.embeddings = nn.Sequential(
            nn.Linear(512*24, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 128),
            nn.ReLU(inplace=True))

        # extract weights from `vggish_list`
        for seq in (self.features, self.embeddings):
            for layer in seq:
                if type(layer).__name__ != "MaxPool2d" and type(layer).__name__ != "ReLU":
                    layer.weight = next(param_generator())
                    layer.bias = next(param_generator())

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.embeddings(x)
        return x

net = VGGish()
net.eval()

# Save weights to disk
torch.save(net.state_dict(), "./vggish.pth")